# ES → BBF Salesforce Service Migration

This notebook migrates Order records from ES Salesforce to Service__c in BBF Salesforce.

## Prerequisites
- **Account migration must be completed first** (Accounts have BBF_New_Id__c populated)
- **BAN__c migration must be completed first** (Billing_Invoice__c has BBF_New_Id__c populated)
- **Location__c migration must be completed first** (Address__c has BBF_New_Id__c populated)
- ES Order records must have BBF_New_Id__c field created

## Object Mapping
- **ES Source:** Order (Salesforce Standard Object with custom fields)
- **BBF Target:** Service__c (Custom Object)

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query Order from ES where:
   - Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
   - Project_Group__c NOT LIKE '%PA MARKET DECOM%'
   - Service_Order_Record_Type__c = 'Service Order Agreement' 
   - Parent BAN (Billing_Invoice__c) has BBF_New_Id__c populated (BAN already migrated)
   - Parent Account has BBF_New_Id__c populated (Account already migrated)
   - Address_A__c has BBF_New_Id__c populated (Location already migrated)
   - BBF_New_Id__c is empty (not yet migrated)
3. Transform ES Order for BBF Service__c schema:
   - Map Billing_Invoice__c → BBF BAN ID via ES Billing_Invoice__r.BBF_New_Id__c (MASTER-DETAIL REQUIRED)
   - Map AccountId → BBF Account ID via ES Account.BBF_New_Id__c
   - Map Address_A__c → BBF Location ID via ES Address__r.BBF_New_Id__c 
   - Map Address_Z__c → BBF Location ID via ES Address__r.BBF_New_Id__c
   - Set required fields: TSP__c = False, busUnit__c = 'EVS'
   - Add ES_Legacy_ID__c = ES Order.Id (for tracking)
4. Insert Service__c to BBF Salesforce
5. Update ES Order with BBF_New_Id__c = BBF Service.Id
6. Create ID mapping: ES Order ID → BBF Service ID
7. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF Service__c:** `ES_Legacy_ID__c` stores original ES Order ID
- Text(18), External ID, Unique

**In ES Order:**
- `BBF_New_Id__c` stores new BBF Service ID after migration (Text 18)

## Safety
- `TEST_MODE = True` by default (limits to 10 Services)
- Only migrates Orders with Status = Activated/Suspended/Disconnect in Progress
- Skips Orders where parent BAN not yet migrated
- Skips Orders where parent Account not yet migrated
- Skips Orders where Location not yet migrated
- Skips Orders already migrated (BBF_New_Id__c populated)
- Skips PA MARKET DECOM orders

In [12]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Set-up successful


In [13]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
# ES_USERNAME = "sfdcapi@everstream.net"
# ES_PASSWORD = "pV4CAxns8DQtJsBq!"
# ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
# ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = False  # ⚠️ Set to False to migrate ALL Services
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 Service Owner - Set all migrated Services to this user
OWNER_ID = "005Ea00000ZOGFZIA5"  # Same as Account/Contact/BAN/Location migration

# 🏢 Default Business Unit - Required picklist field in BBF
# ES records being migrated should be assigned to EVS business unit
DEFAULT_BUS_UNIT = "EVS"  # EverStream business unit in BBF

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_service_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Default Business Unit: {DEFAULT_BUS_UNIT}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

📋 Configuration loaded
   TEST_MODE: False
   Owner ID: 005Ea00000ZOGFZIA5
   Default Business Unit: EVS
   Output: es_bbf_service_migration_20260114_112839.xlsx

⚠️  Note: Bulk API automatically handles batching (200 records/batch)


In [14]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

CONNECTING TO SALESFORCE ORGS

📌 Connecting to ES (source)...
✅ Connected to ES: everstream--uat.sandbox.my.salesforce.com

📌 Connecting to BBF (target)...
✅ Connected to BBF: bluebirdnetwork--full.sandbox.my.salesforce.com


In [15]:
# === QUERY ES ORDER (Services) ===
# Only Order where:
# 1. Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
# 2. Project_Group__c NOT LIKE '%PA MARKET DECOM%'
# 3. Service_Order_Record_Type__c = 'Service Order Agreement'
# 4. Parent BAN has BBF_New_Id__c populated (BAN already migrated)
# 5. Parent Account has BBF_New_Id__c populated (Account already migrated)
# 6. Address_A__c has BBF_New_Id__c populated (Location already migrated)
# 7. Order does NOT have BBF_New_Id__c populated (not yet migrated)

print("\n" + "=" * 80)
print("QUERYING ES ORDER (Services)")
print("=" * 80)

# Build query - filter for actively billing orders only
# Note: Query on single line to avoid SOQL parsing issues with newlines
query = "SELECT Id, Name, OrderNumber, Service_ID__c, Status, Type, Description, AccountId, Account.BBF_New_Id__c, Account.Name, Billing_Invoice__c, Billing_Invoice__r.BBF_New_Id__c, Billing_Invoice__r.Name, Address_A__c, Address_A__r.BBF_New_Id__c, Address_A__r.Name, Address_Z__c, Address_Z__r.BBF_New_Id__c, Address_Z__r.Name, Node__c, Project_Group__c, Service_Order_Record_Type__c, Service_Start_Date__c, Billing_Start_Date__c, Service_Term_Months_Est__c, Contract_Type__c, MRC__c, NRC__c, Bandwidth_mbps__c, Circuit_ID__c, Product__c, OwnerId FROM Order WHERE Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress') AND (Project_Group__c = null OR (NOT Project_Group__c LIKE '%PA MARKET DECOM%')) AND Service_Order_Record_Type__c = 'Service Order Agreement' AND Billing_Invoice__r.BBF_New_Id__c != null AND Billing_Invoice__r.BBF_New_Id__c != '' AND Account.BBF_New_Id__c != null AND Account.BBF_New_Id__c != '' AND Address_A__r.BBF_New_Id__c != null AND Address_A__r.BBF_New_Id__c != '' AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')"

# Add limit for test mode
if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query (first 200 chars): {query[:200]}...")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_orders = result["records"]

print(f"✅ Found {len(es_orders)} Order records to migrate")

if len(es_orders) > 0:
    sample = es_orders[0]
    print(f"\nSample Order:")
    print(f"  ID:                {sample['Id']}")
    print(f"  Name:              {sample.get('Name', 'N/A')}")
    print(f"  Service ID:        {sample.get('Service_ID__c', 'N/A')}")
    print(f"  Status:            {sample.get('Status', 'N/A')}")
    print(f"  ES Account:        {sample.get('AccountId', 'N/A')}")
    print(
        f"  BBF Account (target): {sample.get('Account', {}).get('BBF_New_Id__c', 'N/A')}"
    )
    print(f"  ES BAN:            {sample.get('Billing_Invoice__c', 'N/A')}")
    print(
        f"  BBF BAN (target):  {sample.get('Billing_Invoice__r', {}).get('BBF_New_Id__c', 'N/A')}"
    )
    print(f"  ES Location A:     {sample.get('Address_A__c', 'N/A')}")
    print(
        f"  BBF Location A (target): {sample.get('Address_A__r', {}).get('BBF_New_Id__c', 'N/A')}"
    )
elif TEST_MODE:
    print("\n⚠️  No unmigrated Order records found in test set")
    print("   Check: Are there ES Orders with parent BANs/Accounts/Locations migrated?")
else:
    print("\n✅ All Order records have been migrated (or no eligible records found)!")


QUERYING ES ORDER (Services)
Query (first 200 chars): SELECT Id, Name, OrderNumber, Service_ID__c, Status, Type, Description, AccountId, Account.BBF_New_Id__c, Account.Name, Billing_Invoice__c, Billing_Invoice__r.BBF_New_Id__c, Billing_Invoice__r.Name, A...

Executing query...


SalesforceMalformedRequest: Malformed request https://everstream--uat.sandbox.my.salesforce.com/services/data/v59.0/query/?q=SELECT+Id%2C+Name%2C+OrderNumber%2C+Service_ID__c%2C+Status%2C+Type%2C+Description%2C+AccountId%2C+Account.BBF_New_Id__c%2C+Account.Name%2C+Billing_Invoice__c%2C+Billing_Invoice__r.BBF_New_Id__c%2C+Billing_Invoice__r.Name%2C+Address_A__c%2C+Address_A__r.BBF_New_Id__c%2C+Address_A__r.Name%2C+Address_Z__c%2C+Address_Z__r.BBF_New_Id__c%2C+Address_Z__r.Name%2C+Node__c%2C+Project_Group__c%2C+Service_Order_Record_Type__c%2C+Service_Start_Date__c%2C+Billing_Start_Date__c%2C+Service_Term_Months_Est__c%2C+Contract_Type__c%2C+MRC__c%2C+NRC__c%2C+Bandwidth_mbps__c%2C+Circuit_ID__c%2C+Product__c%2C+OwnerId+FROM+Order+WHERE+Status+IN+%28%27Activated%27%2C+%27Suspended+%28Late+Payment%29%27%2C+%27Disconnect+in+Progress%27%29+AND+%28Project_Group__c+%3D+null+OR+%28NOT+Project_Group__c+LIKE+%27%25PA+MARKET+DECOM%25%27%29%29+AND+Service_Order_Record_Type__c+%3D+%27Service+Order+A

In [ ]:
# === TRANSFORM FOR BBF SERVICE__c ===
# Day 1 Migration: REQUIRED FIELDS ONLY (no booleans - they default to False)

print("\n" + "=" * 80)
print("TRANSFORMING ORDER FOR BBF SERVICE__c (REQUIRED FIELDS ONLY)")
print("=" * 80)

bbf_services = []
skipped_no_bbf_ban = []

for es_order in es_orders:
    # Get the BBF BAN ID from the parent Billing_Invoice__c (MASTER-DETAIL REQUIRED)
    bbf_ban_id = None
    if es_order.get("Billing_Invoice__r") and es_order["Billing_Invoice__r"].get(
        "BBF_New_Id__c"
    ):
        bbf_ban_id = es_order["Billing_Invoice__r"]["BBF_New_Id__c"]

    # Safety check: Skip if no BBF BAN ID (MASTER-DETAIL is BLOCKING)
    if not bbf_ban_id:
        skipped_no_bbf_ban.append(
            {
                "es_id": es_order["Id"],
                "name": es_order.get("Name", "N/A"),
                "es_ban_id": es_order.get("Billing_Invoice__c"),
                "reason": "No BBF BAN ID found - Master-Detail is required",
            }
        )
        continue

    # Build service name from available fields
    service_name = (
        es_order.get("Service_ID__c")
        or es_order.get("Name")
        or es_order.get("OrderNumber")
        or "Unknown Service"
    )

    # =========================================================================
    # BBF Service__c - REQUIRED FIELDS ONLY (Day 1 Migration)
    # Booleans (TSP__c) default to False - no need to set
    # =========================================================================
    bbf_service = {
        # 🔴 REQUIRED: Name (max 80 chars)
        "Name": service_name[:80],
        
        # 🔴 REQUIRED: Master-Detail to BAN__c
        "Billing_Account_Number__c": bbf_ban_id,
        
        # 👤 Owner
        "OwnerId": OWNER_ID,
        
        # 🔗 Tracking field
        "ES_Legacy_ID__c": es_order["Id"],
    }

    bbf_services.append(bbf_service)

print(f"✅ Transformed {len(bbf_services)} Services")
print(f"\n   REQUIRED FIELDS ONLY:")
print(f"   - Name (from Service_ID__c/Name/OrderNumber)")
print(f"   - Billing_Account_Number__c (Master-Detail to BAN__c)")
print(f"   - OwnerId = {OWNER_ID}")
print(f"   - ES_Legacy_ID__c (tracking)")
print(f"\n   Note: TSP__c defaults to False - not explicitly set")

if len(skipped_no_bbf_ban) > 0:
    print(f"\n⚠️  Skipped {len(skipped_no_bbf_ban)} Services (no BBF BAN ID - BLOCKING)")
    for skip in skipped_no_bbf_ban[:5]:
        print(f"   - {skip['name']} (ES BAN: {skip['es_ban_id']})")

In [ ]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING SERVICES TO BBF")
print("=" * 80)

if len(bbf_services) == 0:
    print("⚠️  No Services to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_services)} Services using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Service__c.insert(bbf_services)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_services[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "name": bbf_services[i].get("Name", "N/A"),
                        "bbf_ban_id": bbf_services[i]["Billing_Account_Number__c"],
                        "bbf_account_id": bbf_services[i]["Account__c"],
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_services[i]["ES_Legacy_ID__c"],
                        "name": bbf_services[i].get("Name", "N/A"),
                        "errors": result["errors"],
                        "bbf_ban_id": bbf_services[i]["Billing_Account_Number__c"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} Services")
        print(f"❌ Failed to insert: {len(failed_inserts)} Services")

        if len(failed_inserts) > 0:
            print(f"\nFailed Services (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - {item['name']} (ES ID: {item['es_id']})")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES Order ID:     {sample['es_id']}")
            print(f"  BBF Service ID:  {sample['bbf_id']}")
            print(f"  Name:            {sample['name']}")
            print(f"  BBF BAN ID:      {sample['bbf_ban_id']}")
            print(f"  BBF Account ID:  {sample['bbf_account_id']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []

In [ ]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No Services to update in ES")
    es_update_results = []
else:
    # Build update records for ES - set BBF_New_Id__c only
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    print(f"Updating {len(es_updates)} Order records in ES...")
    print("   - Setting BBF_New_Id__c = BBF Service ID")

    try:
        es_update_results = es_sf.bulk.Order.update(es_updates)

        success_count = sum(1 for r in es_update_results if r["success"])
        error_count = sum(1 for r in es_update_results if not r["success"])

        print(f"\n✅ Successfully updated: {success_count} Order records in ES")
        print(f"❌ Failed to update: {error_count} Order records in ES")

        if error_count > 0:
            print("\nFirst 10 update failures:")
            fail_count = 0
            for i, r in enumerate(es_update_results):
                if not r["success"] and fail_count < 10:
                    print(f"  - {es_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1

    except Exception as e:
        print(f"❌ Error during ES update: {e}")
        es_update_results = []

In [ ]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Status colors
status_colors = {
    "Success": "C6EFCE",  # Green
    "Failed": "FFC7CE",  # Red
    "Skipped": "FFEB9C",  # Yellow
}

# --- SHEET 1: Migration Results ---
headers1 = [
    "ES Order ID",
    "BBF Service ID",
    "Service Name",
    "BBF BAN ID",
    "BBF Account ID",
    "Status",
    "Error",
]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Combine all results
all_results = []
for item in successful_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": item["bbf_id"],
            "Name": item["name"],
            "BBF_BAN_ID": item["bbf_ban_id"],
            "BBF_Account_ID": item["bbf_account_id"],
            "Status": "Success",
            "Error": "",
        }
    )
for item in failed_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_BAN_ID": item["bbf_ban_id"],
            "BBF_Account_ID": "",
            "Status": "Failed",
            "Error": str(item["errors"]),
        }
    )
for item in skipped_no_bbf_ban:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_BAN_ID": "",
            "BBF_Account_ID": "",
            "Status": "Skipped",
            "Error": "No BBF BAN ID found (Master-Detail required)",
        }
    )
for item in skipped_no_bbf_account:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_BAN_ID": "",
            "BBF_Account_ID": "",
            "Status": "Skipped",
            "Error": "No BBF Account ID found",
        }
    )
for item in skipped_no_bbf_location:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_BAN_ID": "",
            "BBF_Account_ID": "",
            "Status": "Skipped",
            "Error": "No BBF Location A ID found",
        }
    )

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_ID"],
            r["BBF_ID"],
            r["Name"],
            r["BBF_BAN_ID"],
            r["BBF_Account_ID"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Service Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append(["Business Unit:", DEFAULT_BUS_UNIT])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A8"].font = Font(bold=True)
ws2["B8"].font = Font(bold=True)
ws2.append(["Total Services Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Skipped (No BBF BAN)", len(skipped_no_bbf_ban)])
ws2.append(["Skipped (No BBF Account)", len(skipped_no_bbf_account)])
ws2.append(["Skipped (No BBF Location)", len(skipped_no_bbf_location)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = [
    "ES Order ID",
    "BBF Service ID",
    "BBF BAN ID",
    "BBF Account ID",
    "Service Name",
]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append(
        [
            item["es_id"],
            item["bbf_id"],
            item["bbf_ban_id"],
            item["bbf_account_id"],
            item["name"],
        ]
    )

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Order ID", "Service Name", "BBF BAN ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            item["bbf_ban_id"],
            str(item["errors"]),
        ]
    )

# Also add skipped records to failed inserts for tracking
for item in skipped_no_bbf_ban:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            "",
            "Skipped: No BBF BAN ID found (Master-Detail required)",
        ]
    )

for item in skipped_no_bbf_account:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            "",
            "Skipped: No BBF Account ID found",
        ]
    )

for item in skipped_no_bbf_location:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            "",
            "Skipped: No BBF Location A ID found",
        ]
    )

for col in ws4.columns:
    col_cells = list(col)
    max_length = max(len(str(cell.value)) for cell in col_cells) if col_cells else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} Services, color-coded)")
print(f"   📈 Sheet 2: Summary (metrics and stats)")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} successful mappings)")
print(
    f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts) + len(skipped_no_bbf_ban) + len(skipped_no_bbf_account) + len(skipped_no_bbf_location)} failures)"
)

In [ ]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Order queried: {len(es_orders)}")
print(f"BBF Service__c inserted: {len(successful_inserts)}")
print(
    f"Success rate: {len(successful_inserts)/len(es_orders)*100:.1f}%"
    if len(es_orders) > 0
    else "N/A - No Services processed"
)
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print(f"\n🔄 TEST MODE complete. Only migrated {TEST_LIMIT} Services.")
    print("   To migrate ALL Services, set TEST_MODE = False in Cell 2 and re-run.")
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Service migration finished.")
    print("   Next: Migrate Service_Charge__c (OrderItem → Service_Charge__c)")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} Services failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")

if len(skipped_no_bbf_ban) > 0:
    print(
        f"\n⚠️  {len(skipped_no_bbf_ban)} Services skipped (no BBF BAN - MASTER-DETAIL REQUIRED)"
    )
    print("   These Orders must have their parent BAN migrated first")

---
## Next Steps: Service_Charge__c Migration

After Service migration is complete, use the **ID Mapping sheet** from this Excel output to migrate Service_Charge__c:

### Prerequisites for Service_Charge__c Migration
| Prerequisite | Status | Notes |
|--------------|--------|-------|
| Account | ✅ Complete | Account migration done |
| BAN__c | ✅ Complete | BAN migration done |
| Contact | ✅ Complete | Contact migration done |
| Location__c | ✅ Complete | Location migration done |
| Service__c | 🔄 This notebook | Use ID Mapping sheet |

### ID Mapping Files Needed for Service_Charge__c
- `es_bbf_service_migration_*.xlsx` → Service ID mapping (this file)

### ES OrderItem → BBF Service_Charge__c
- Master-Detail: Service_Charge__c.Service__c → Service__c.Id
- Query: OrderItem WHERE Order.BBF_New_Id__c != null
- Map: OrderItem.OrderId → ES Order.BBF_New_Id__c (BBF Service ID)

## Field Mapping Reference

### ES Order → BBF Service__c Field Mapping (Day 1 Minimum)

| ES Field | BBF Field | Notes |
|----------|-----------|-------|
| Id | ES_Legacy_ID__c | Tracking |
| Service_ID__c / Name / OrderNumber | Name | Service identifier |
| Billing_Invoice__r.BBF_New_Id__c | Billing_Account_Number__c | MASTER-DETAIL (REQUIRED) |
| Account.BBF_New_Id__c | Account__c | Lookup to Account |
| Address_A__r.BBF_New_Id__c | A_Location__c | Lookup to Location |
| Address_Z__r.BBF_New_Id__c | Z_Location__c | Lookup to Location (optional) |
| (default False) | TSP__c | Required boolean |
| (default 'EVS') | busUnit__c | Required picklist |
| Status | Service_Status__c | Picklist mapping |
| Description | Service_Description__c | Text |
| Circuit_ID__c | Circuit_ID__c | Direct map |
| Billing_Start_Date__c | Billing_Start_Date__c | Date |
| MRC__c | MRC__c | Currency |
| NRC__c | NRC__c | Currency |
| Bandwidth_mbps__c | Bandwidth__c | Number |
| Term_Months__c | Contract_Term_Months__c | Number |

## Cleanup Apex (if needed)

### Delete Migrated Services from BBF
```apex
List<Service__c> services = [SELECT Id, Name FROM Service__c WHERE ES_Legacy_ID__c != null];
System.debug('Found ' + services.size() + ' migrated Services');
delete services;
```

### Remove BBF_New_Id__c from ES (to re-run migration)
```apex
List<Order> orders = [SELECT Id, BBF_New_Id__c 
                      FROM Order 
                      WHERE BBF_New_Id__c != NULL];
System.debug('Found ' + orders.size() + ' records to reset');
for (Order ord : orders) {
    ord.BBF_New_Id__c = NULL;
}
update orders;
```